**`namespace`** highest member in the heirarchy of method calling
Basic syntax is `namespace.class.method`
The `using` keyword can be invoked so that the namespace is not required
*Example*: 

`using System; Console.WriteLine("Hello")`

is the same as

`
System.Console.WriteLine("Hello");
`

In [1]:
//`operation` acts much like a function in python
operation SayHello(name: String) : Unit{
    Message($"Hello {name}");
}

SayHello

*NB* Syntax for formatting strings with operation params
`$"{param}"`

In [2]:
//after running cell, Q# compiles code and returns the names of any operations it foundin the cell
//compiler will also return any errors it found
operation SayHelloQ() : Unit{
    SayHello("Q");
}

SayHelloQ

**(?)** Adding comments after a closing curly brace throws an error (seen below)
`/snippet_.qs(3,13): error QS1002: An opening bracket has not been closed.`

In [3]:

//operation Foo() : Unit{
//
//}//comment


In [4]:
//importing math module
open Microsoft.Quantum.Math;

//call SayHello on PI from Microsoft.Quantum.Math namespace
operation SayHelloPI() : Unit{
    let pi = Microsoft.Quantum.Convert.DoubleAsString(PI());
    SayHello(pi);
}


SayHelloPI

In [5]:
open Microsoft.Quantum.Math;

//set the quibt state to |+>
operation SetPlus(q:Qubit) : Unit{
    //put the quibt into |0>
    Reset(q);
    H(q);
}

//set the quibt state to |->
//H|1> = |->
operation SetMinus(q:Qubit) : Unit{
    //put the quibt into |0>
    Reset(q);
    //put the qubit into |1>
    X(q);
    H(q);
}

//randomly set the qubit into |+> or |->
operation PrepareRandomMessage(q:Qubit) : Unit{
    let choice = RandomInt(2);
    
    if (choice == 0){
        Message("Prepared |->");
        SetMinus(q);
    } else{
        Message("Prepared |+>");
        SetPlus(q);
    }
}

PrepareRandomMessage, SetMinus, SetPlus

In [6]:
//continued use of the operations just created
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Measurement;

operation NextRandomBit() : Result{
    //instantiate a quibt
    using (q=Qubit()){
        SetPlus(q);
        //measure qubit in the z basis then reset it to |0>
        return MResetZ(q);
    }
}

operation TestPrepareQubits() : Result{
    //like a NoneType in Python
    mutable r = Zero;
    
    //prepare 5 identical qubits in |0>
    using (qubits = Qubit[5]){
        ApplyToEach(PrepareRandomMessage, qubits);
        //generate output to the console (since params are empty)
        DumpMachine();
        
        //measure each qubit in the pauli x basis
        set r = Measure([PauliX, PauliX, PauliX, PauliX, PauliX], qubits);
    
        //measure the array of qubits and ensure they are |0> to be safely released
        ResetAll(qubits);
    }
    
    //return the array of measurements
    return r;
    
}

NextRandomBit, TestPrepareQubits

After operations compile, use `%simulate` to simulate the operation

`%simulate` prints any console output to screen
print `()` if it encounters `Unit`
Otherwise it will print the actual return value

In [7]:
%simulate SayHelloPI

Hello 3.141592653589793


()

In [8]:
%simulate NextRandomBit

One

Since `%simulate` *only* accepts operations (does not accept arguments) you should create a wrapper operation to call `%simulate` with an argument. 

For example, you cannot call `%simulate SayHello` because `SayHello` requires an argument `name`
However you can call `%simulate SayHelloQ` because:
- `SayHelloQ` is a wrapper operation
- within the body of `SayHelloQ` you supplied `name` to `SayHello`

In [9]:
%simulate TestPrepareQubits

Prepared |->
Prepared |+>
Prepared |+>
Prepared |->
Prepared |+>


|0⟩	0.17677669529663692 + 0𝑖
|1⟩	-0.17677669529663692 + 0𝑖
|2⟩	0.17677669529663692 + 0𝑖
|3⟩	-0.17677669529663692 + 0𝑖
|4⟩	0.17677669529663692 + 0𝑖
|5⟩	-0.17677669529663692 + 0𝑖
|6⟩	0.17677669529663692 + 0𝑖
|7⟩	-0.17677669529663692 + 0𝑖
|8⟩	-0.17677669529663692 + 0𝑖
|9⟩	0.17677669529663692 + 0𝑖
|10⟩	-0.17677669529663692 + 0𝑖
|11⟩	0.17677669529663692 + 0𝑖
|12⟩	-0.17677669529663692 + 0𝑖
|13⟩	0.17677669529663692 + 0𝑖
|14⟩	-0.17677669529663692 + 0𝑖
|15⟩	0.17677669529663692 + 0𝑖
|16⟩	0.17677669529663692 + 0𝑖
|17⟩	-0.17677669529663692 + 0𝑖
|18⟩	0.17677669529663692 + 0𝑖
|19⟩	-0.17677669529663692 + 0𝑖
|20⟩	0.17677669529663692 + 0𝑖
|21⟩	-0.17677669529663692 + 0𝑖
|22⟩	0.17677669529663692 + 0𝑖
|23⟩	-0.17677669529663692 + 0𝑖
|24⟩	-0.17677669529663692 + 0𝑖
|25⟩	0.17677669529663692 + 0𝑖
|26⟩	-0.17677669529663692 + 0𝑖
|27⟩	0.17677669529663692 + 0𝑖
|28⟩	-0.17677669529663692 + 0𝑖
|29⟩	0.17677669529663692 + 0𝑖
|30⟩	-0.17677669529663692 + 0𝑖
|31⟩	0.17677669529663692 + 0𝑖

Zero

Estimating the number of resources (gates) a no-arguments operation will take using `%estimate`

In [10]:
%estimate TestPrepareQubits

Metric,Sum
CNOT,0
QubitClifford,10
R,0
Measure,11
T,0
Depth,0
Width,5
BorrowedWidth,0


In [11]:
%workspace

The notebook "knows about" all .qs files in the same directory as it<br>
It will try to compile all of these .qs files<br>
Then, the operations defined elsewhere can be used here in the notebook<br>
To pick up any changes you make to a Q# file in the workspace, use `%workspace reload`<br>

### Loading the docstring of an operation ###
Add `?` after the operation name<br>
Notice the syntax highlighting<br>
For example

In [12]:
Microsoft.Quantum.Intrinsic.Z?

Microsoft.Quantum.Intrinsic.Z

You can add your own docstrings using XML tags<br>
Here is an example 

In [13]:
/// # Summary #
///An example of user added documentation
/// # Input #
/// There is no input
operation MyExample() : Unit{
    Message("");
}

MyExample

Now, if we try `MyExample?` we should see our summary

In [14]:
MyExample?

MyExample

`%who` returns all **local** and **workspace** operations available

In [15]:
%who

MyExample, NextRandomBit, PrepareRandomMessage, SayHello, SayHelloPI, SayHelloQ, SetMinus, SetPlus, TestPrepareQubits